# DETECTOR DE SPAM PARA CORREOS
### PROFESOR
- ZURIEL DATHAN MORA FELIX 
### INTEGRANTES 
- AMARILLAS AVILES BRAYAN ALEXIS 
- CUEN ARMENTA ALMA VICTORIA 


In [3]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

import tkinter as tk
from tkinter import messagebox


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Brayan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. IMPORTAR  DATASET

In [4]:
dataset = pd.read_csv("spam_assassin.csv")
dataset.head()

,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


## 2. PROCESAMIENTO DE DATOS 

In [5]:
#Eliminar correos duplicados
ds_clean = dataset.drop_duplicates(subset="text").copy()

#Convertir a minúsculas
ds_clean['text'] = ds_clean['text'].str.lower()

#Eliminar caracteres especiales
ds_clean['text'] = ds_clean['text'].str.replace('[^a-z0-9\\s]', '', regex=True)

#Eliminar palabras vacías y stopwords
stop_words = set(stopwords.words('english'))
ds_clean['text'] = ds_clean['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

#Tokenizar el texto
ds_clean['text'] = ds_clean['text'].apply(lambda x: x.split())


## 3. PREPARACIÓN DEL MODELO 


In [6]:

# Dividir el dataset en conjunto de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(ds_clean['text'].apply(lambda x: ' '.join(x)), ds_clean['target'], test_size=0.20, random_state=42)

# 2. Inicializar CountVectorizer
vectorizer = CountVectorizer() 

# Ajustar y transformar el conjunto de entrenamiento
x_train_tfidf = vectorizer.fit_transform(x_train)


## ALGORITMO DE BAYES

In [7]:
# Inicializar el clasificador Bayes
nb = MultinomialNB()

# Ajustar el clasificador con el conjunto de entrenamiento
nb.fit(x_train_tfidf, y_train)

# Prueba del modelo

emails = [
    "Congratulations! You've won a free iPhone! Click here to claim your prize.",
    "Act now and get a limited time offer of 90% off on all products! Hurry, offer expires soon!",
    "Earn thousands of dollars from home! No experience required, sign up now!",
    "You have a message from someone you know! Click here to view it.",
    "Buy cheap Viagra from our website today. Don't miss out on this amazing deal!",
    "Don't miss this opportunity! Free vacation for you and a friend, just fill out this form!",
    "Get rich fast by investing in this exclusive cryptocurrency! Join now and make a fortune.",
    "Hurry! You’re almost out of time to claim your free gift card worth $500! Click here!",
    "Your bank account has been compromised. Click here to reset your password immediately.",
    "Join our webinar to learn how to make money fast with zero investment.",
    "Dear customer, your order has been shipped. Thank you for shopping with us!",
    "Hello! Just checking in to see how you’re doing with the new software. Let me know if you need any assistance.",
    "Reminder: Your appointment with Dr. Smith is scheduled for 10 AM tomorrow.",
    "Hi, I wanted to confirm our meeting for this afternoon. Please let me know if you need to reschedule.",
    "You’re receiving this email because you signed up for our newsletter. Here's our latest update.",
    "Your subscription is about to renew. If you wish to make any changes, please visit your account settings.",
    "Thank you for your purchase! Your receipt is attached for your reference.",
    "Your payment has been successfully processed. Thank you for choosing our services.",
    "Hi John, we missed you at the meeting. Here’s the summary of what was discussed.",
    "You’ve been invited to attend a conference call. The details are attached."
]

emails_count = vectorizer.transform(emails)
predictions = nb.predict(emails_count)
print(predictions)


[1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 0]


## INTERFAZ

In [8]:
def validar():
    email = texto_cuerpo.get("1.0", "end-1c")
    emails = [email]
    emails_count = vectorizer.transform(emails)
    predictions = nb.predict(emails_count)
    if predictions[0] == 1:
        messagebox.showinfo("Resultado", "El correo es spam")
    else:
        messagebox.showinfo("Resultado", "El correo no es spam")

# Crear la ventana principal
root = tk.Tk()
root.title("Enviar Correo")
root.geometry("400x300")

# Área de texto para el cuerpo del correo
tk.Label(root, text="Cuerpo del correo:").pack(anchor="w", padx=10, pady=5)
texto_cuerpo = tk.Text(root, height=5, width=50)
texto_cuerpo.pack(padx=10)

# Botón de enviar
boton_enviar = tk.Button(root, text="Enviar" , command=validar)
boton_enviar.pack(pady=10)

# Iniciar la aplicación
root.mainloop()


## CALCULAR LA PRECISION DEL MODELO

In [9]:
x_test_count = vectorizer.transform(x_test)
nb.score(x_test_count, y_test)

0.9681050656660413

## CLASIFICACION DE SPAM

In [10]:
# Calcular la probabilidad previa de spam (P(Spam))
p_spam = np.sum(ds_clean['target']) / len(ds_clean['target'])

#Probabilidad previa de no spam
p_no_spam = 1 - p_spam

x_train_array = x_train_tfidf.toarray()
# Calcular la probabilidad de las caracteristicas del correo electrocnico dado que es spam (P(Correo|Spam))
p_caracteristicas_spam = x_train_array[y_train == 1].sum(axis=0) / x_train_array[y_train == 1].sum()
# Calcular la probabilidad de las caracteristicas del correo electronico dado que no es spam (P(Correo|No Spam))
p_caracteristicas_no_spam = x_train_array[y_train == 0].sum(axis=0) / x_train_array[y_train == 0].sum()

# Calcular la probabilidad posterior de que el correo electronico sea spam (P(Spam|Caracteristicas))
p_spam_caracteristicas = (p_spam * p_caracteristicas_spam) / (p_spam * p_caracteristicas_spam + p_no_spam * p_caracteristicas_no_spam)

# Un correo se clasificara como spam si P(Spam|Caracteristicas) > P(No Spam|Caracteristicas)

# Clasificamos los correos del conjunto de prueba
clasificacion = []
for i in range(x_test_count.shape[0]):
    probabilidad_condicional_spam = (p_spam * x_test_count[i].multiply(p_caracteristicas_spam)).sum()
    probabilidad_condicional_no_spam = (p_no_spam * x_test_count[i].multiply(p_caracteristicas_no_spam)).sum()
    
    probabilidad_posterior_spam = probabilidad_condicional_spam / (probabilidad_condicional_spam + probabilidad_condicional_no_spam)
    # Si la probabilidad otorga >0.5 es spam, <0.5 no es spam
    clasificacion.append(1 if probabilidad_posterior_spam > 0.5 else 0)

# Convertimos las clasificaciones a un arreglo de numpy
clasificacion = np.array(clasificacion)

#Evaluacion
precision = np.sum(clasificacion == y_test.values) / len(clasificacion)
recuperacion = np.sum(clasificacion[y_test.values == 1] == y_test.values[y_test.values == 1]) / y_test.values.sum()

print("Presicion: ", precision)
print("Recuperacion: ", recuperacion)



Presicion:  0.702626641651032
Recuperacion:  0.06764705882352941
